<a href="https://colab.research.google.com/github/DinhoVCO/Mineria-de-datos/blob/main/Final_ALGORITMO_APRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ALGORITMO APRIORI**
---
**UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**


> ESCUELA PROFESIONAL: Ing. Informatica y de Sistemas



> CURSO: Minería de datos💻


> ALUMNO : Vega Centeno Olivera, Ronaldinho🧑









##**1.-Instalar bibliotecas**

In [315]:
import numpy as np #para procesar el dataset
import statistics as stats #para hallar medidas estadisticas

## **2.-Importar datos**

In [316]:
#Importamos el archivo .npy en la variable data de tipo numpy.ndarray 0-dimensional
data = np.load('spotify.npy',allow_pickle=True,fix_imports=True)
#convertimos a una lista -> obtenemos un diccionario
data=data.tolist()
#Cantidad de playlist
print("Total de PlayLists:",len(data))

Total de PlayLists: 10000


In [318]:
#Canciones de la playlist n° 0 sin repeticiones y ordenadas alfabeticamente
set(data[0][0:5])

{'Crazy In Love',
 "It Wasn't Me",
 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'Rock Your Body',
 'Toxic'}

##**3.- Preprocesar (lista de listas)**

In [323]:
#Generamos una lista de canciones unicas y de playlist sin canciones repetidas
def dataset_preprocessing(data):
  playlist=[]
  songs=set()
  for i in data:
    playlist.append(set(data[i]))
    songs|=set(data[i])
  
  songs=[[i] for i in list(songs)]
  songs.sort()
  return list(map(frozenset,songs)), playlist

In [324]:
songs, playlist = dataset_preprocessing(data)
print('Total de canciones distintas:', len(songs))
print('Total de Playlists(transaccciones):', len(playlist))

Total de canciones distintas: 132920
Total de Playlists(transaccciones): 10000


In [325]:
#Tres canciones
songs[:3]

[frozenset({'!Dios Mio! El Diablo Es Muy Picante'}),
 frozenset({'"*!*!" What U Heard'}),
 frozenset({'"2014: A Pesach Medley"'})]

In [326]:
#Plailist n°999
playlist[999]

{'A Quién Le Importa',
 'Amor De Pobre - No Tengo Dinero (Hip-Hop Version)',
 'El Hombre Es Como El Oso',
 'El Mario de Tu Mujer (feat. Don Miguelo)',
 'El Regeton',
 'El Tao Tao',
 'Juana La Cubana',
 'La Bomba',
 'La Mesa Que Mas Aplauda (Za Za Za)',
 'La Vaca',
 'Muévelo',
 'Party In Miami',
 'Que Lo Que',
 'Tu Pun Pun',
 'Un Ladrón'}

## **4.-Definir umbrales** 🟠⏮ 

In [327]:
frequencies=dict()
for i in playlist:
  for j in i:
    frequencies[j] =frequencies.get(j, 0) + 1
frecuencias=list(frequencies.values())
frecuencias.sort()
print(frecuencias)
print(len(frecuencias))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [328]:
print("Media aritmética: ",stats.mean(frecuencias))
print("Media armónica: ",stats.harmonic_mean(frecuencias)) 
print("Mediana: ",stats.median(frecuencias))
print("Mediana baja: ",stats.median_low(frecuencias))
print("Mediana alta: ",stats.median_high(frecuencias))
print("Percentil 50:",stats.median_grouped(frecuencias)) 
print("Moda: ",stats.mode(frecuencias)) 
print("Varianza poblacional : ",stats.pvariance(frecuencias)) 
print("Desviación estándar: ",stats.pstdev(frecuencias))
print("Varianza muestral :",stats.variance(frecuencias)) 

Media aritmética:  4.910028588624737
Media armónica:  1.4028898793768465
Mediana:  1.0
Mediana baja:  1
Mediana alta:  1
Percentil 50: 1.3384321344317307
Moda:  1
Varianza poblacional :  254.75020336963107
Desviación estándar:  15.960896070384992
Varianza muestral : 254.75211995193587


In [329]:
lon_min = 2 # Número mínimo de productos a considerar en las reglas
veces_minimas_aparece_en_lista=100
soporte_min = veces_minimas_aparece_en_lista / len(playlist)
confianza_min =0.5
sustento_min =1.5

## **5.-Ejecutar algoritmo** 

### **5.1. Get frequent items**

#### get_first_frequent_items

In [370]:
#Modulo optimo para obtener 1-itemsetfrequents
def get_first_frequent_items(transactions, k_itemSet,minSupport):
  frequencies=dict()
  for i in transactions:
    for j in i:
      frequencies[j] =frequencies.get(j, 0) + 1
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
          #ItemSetFrequent.append(key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  ItemSetFrequent=[frozenset({i}) for i in ItemSetFrequent ]
  return ItemSetFrequent, supportItemSet

In [371]:
ItemSetFrequent_1,support = get_first_frequent_items(playlist[:2],songs,0.015)
print( 'Cantidad de  1-ItemSetFrequent: ', len(ItemSetFrequent_1))
print('Cantidad de support hallados:',len(support))
print('Canciones: ', ItemSetFrequent_1[:5])
print("Support de '", support)


Cantidad de  1-ItemSetFrequent:  224
Cantidad de support hallados: 224
Canciones:  [frozenset({'A Million Miles Away'}), frozenset({'A Star Is Born'}), frozenset({'A Thousand Miles'}), frozenset({'A True Hero / A Star Is Born'}), frozenset({'A Whole New World'})]
Support de ' {'Everytime We Touch - Radio Edit': 0.5, 'Right Where You Want Me - Radio Edit Version': 0.5, 'Check Yes Juliet': 0.5, 'Lip Gloss': 0.5, 'How Do You Sleep? - Featuring Ludacris': 0.5, 'Say My Name': 0.5, 'Toxic': 0.5, 'Yo (Excuse Me Miss)': 0.5, 'Crazy In Love': 0.5, 'Beautiful Soul': 0.5, 'Whatcha Say': 0.5, 'Just The Girl': 0.5, 'Dirty Little Secret': 0.5, 'Sk8er Boi': 0.5, 'Hey Ya! - Radio Mix / Club Mix': 0.5, 'Year 3000': 0.5, 'The Great Escape': 0.5, 'Replay': 0.5, 'Mr. Brightside': 0.5, 'Run It!': 0.5, 'Yeah!': 0.5, 'All The Small Things': 0.5, 'A Thousand Miles': 0.5, 'My Boo': 0.5, 'My Happy Ending': 0.5, 'Your Love Is My Drug': 0.5, 'Baby': 0.5, 'Where Is The Love?': 0.5, 'Buttons': 0.5, 'Forever': 0.5, 

#### get_frequent_items

In [330]:
def get_frequent_items(transactions, k_itemSet,minSupport):
  frequencies=dict()
  for i in transactions:
    for j in k_itemSet:
      if (j<=i):
        frequencies[j] =frequencies.get(j, 0) + 1
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
          #ItemSetFrequent.append(key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  return ItemSetFrequent, supportItemSet
  


In [373]:
frec,k = get_first_frequent_items(D[:2], C1, 0.015)
print(len(frec))
print(len(k))
print(k)

224
224
{'Everytime We Touch - Radio Edit': 0.5, 'Right Where You Want Me - Radio Edit Version': 0.5, 'Check Yes Juliet': 0.5, 'Lip Gloss': 0.5, 'How Do You Sleep? - Featuring Ludacris': 0.5, 'Say My Name': 0.5, 'Toxic': 0.5, 'Yo (Excuse Me Miss)': 0.5, 'Crazy In Love': 0.5, 'Beautiful Soul': 0.5, 'Whatcha Say': 0.5, 'Just The Girl': 0.5, 'Dirty Little Secret': 0.5, 'Sk8er Boi': 0.5, 'Hey Ya! - Radio Mix / Club Mix': 0.5, 'Year 3000': 0.5, 'The Great Escape': 0.5, 'Replay': 0.5, 'Mr. Brightside': 0.5, 'Run It!': 0.5, 'Yeah!': 0.5, 'All The Small Things': 0.5, 'A Thousand Miles': 0.5, 'My Boo': 0.5, 'My Happy Ending': 0.5, 'Your Love Is My Drug': 0.5, 'Baby': 0.5, 'Where Is The Love?': 0.5, 'Buttons': 0.5, 'Forever': 0.5, "Stacy's Mom": 0.5, 'Party In The U.S.A.': 0.5, 'Ice Box': 0.5, "Jumpin', Jumpin'": 0.5, 'Lose Control (feat. Ciara & Fat Man Scoop)': 0.5, 'Me & U': 0.5, 'Miss Independent': 0.5, 'Check On It - feat. Bun B and Slim Thug': 0.5, "Leavin'": 0.5, 'See You Again': 0.5, 'Cl

In [378]:
frec,k = get_frequent_items(D[:2], C1, 0.001)
print(len(frec))
print(len(k))
print(k)
print(k)

224
224
{frozenset({'A Thousand Miles'}): 0.5, frozenset({'All The Small Things'}): 0.5, frozenset({'Baby'}): 0.5, frozenset({'Beautiful Soul'}): 0.5, frozenset({'Beep'}): 0.5, frozenset({'Buttons'}): 0.5, frozenset({'Check On It - feat. Bun B and Slim Thug'}): 0.5, frozenset({'Check Yes Juliet'}): 0.5, frozenset({'Closer'}): 0.5, frozenset({'Crazy In Love'}): 0.5, frozenset({'Dirty Little Secret'}): 0.5, frozenset({'Everytime We Touch - Radio Edit'}): 0.5, frozenset({'Forever'}): 0.5, frozenset({'Hey Ya! - Radio Mix / Club Mix'}): 0.5, frozenset({'How Do You Sleep? - Featuring Ludacris'}): 0.5, frozenset({'Ice Box'}): 0.5, frozenset({"It Wasn't Me"}): 0.5, frozenset({"Jumpin', Jumpin'"}): 0.5, frozenset({'Just The Girl'}): 0.5, frozenset({"Leavin'"}): 0.5, frozenset({'Lip Gloss'}): 0.5, frozenset({'Livin on Sunday'}): 0.5, frozenset({'Lose Control (feat. Ciara & Fat Man Scoop)'}): 0.5, frozenset({'Me & U'}): 0.5, frozenset({'Miss Independent'}): 0.5, frozenset({'Mr. Brightside'}): 0.5

### **5.2 Get Combination Apriori**

In [334]:
#GEnerar posibles conjuntos (k-ItemSet) a partir de mezclar los k-1 ItemSetFrequent
#Los ItemSet deben estar ordenados lexicográficamente
# Solo mezclamos los pares de itemsets que difieren en su último ítem.
def get_combination_apriori(ItemSetFrequent, k):
  ItemSet_k=[]
  tam=len(ItemSetFrequent)
  for i in range(tam):
    for j in range(i+1,tam):
      Arr1=list(ItemSetFrequent[i])[:k-1]
      Arr2=list(ItemSetFrequent[j])[:k-1]
      if(Arr1==Arr2):
        ItemSet_k.append(ItemSetFrequent[i]|ItemSetFrequent[j])
  return ItemSet_k


In [336]:
C1, D = dataset_preprocessing(data)
print("Canciones:",len(C1))
print("playlists: ",len(D))
L1, supportData = get_first_frequent_items(D, C1, 0.015)
print(len(L1))
len(L1[:2])
len(C1)

Canciones: 132920
playlists:  10000
349


132920

In [337]:
L1, supportData = get_first_frequent_items(D, C1, 0.015)
print(len(L1))
len(L1[:2])

349


2

In [338]:
L=[L1]
Ck=get_combination_apriori(L[0],1)
len(Ck)

60726

In [340]:
print(Ck[:3])

[frozenset({'0 To 100 / The Catch Up', "'Till I Collapse"}), frozenset({'1-800-273-8255', "'Till I Collapse"}), frozenset({"'Till I Collapse", '24K Magic'})]


### 5.3 Generacion de los k-itemSetsFrecuentes

In [342]:
def get_frequent_itemsets(data, minSupport=0.2):  # Obtenga [L1, L2, L3 ..] lista de conjuntos de elementos frecuentes haciendo un bucle
    songs, playlists = dataset_preprocessing(data)
    L1, supportData = get_first_frequent_items(playlists, songs, minSupport)
    L = [L1]
    print("Total de canciones:",len(L[0]))
    k = 1
    while (len(L[k - 1]) > 0):  # Crear Ck
        print("k==",k," cantidad:",len(L[k-1]))
        Ck = get_combination_apriori(L[k - 1], k)
        print("Ck Generado:",len(Ck))
        Lk, supK = get_frequent_items(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [347]:
L, supportData= get_frequent_itemsets(data,0.01)

Total de canciones: 838
k== 1  cantidad: 838
Ck Generado: 350703
k== 2  cantidad: 167
Ck Generado: 732
k== 3  cantidad: 19
Ck Generado: 17


In [406]:
print(L[0])

[frozenset({"'Till I Collapse"}), frozenset({'0 To 100 / The Catch Up'}), frozenset({'1-800-273-8255'}), frozenset({'10 Bands'}), frozenset({'1985'}), frozenset({'2 Phones'}), frozenset({'24K Magic'}), frozenset({'2U (feat. Justin Bieber)'}), frozenset({'4 AM'}), frozenset({'6 Foot 7 Foot'}), frozenset({'679 (feat. Remy Boyz)'}), frozenset({'7 Years'}), frozenset({'7/11'}), frozenset({'A Milli'}), frozenset({'A Sky Full of Stars'}), frozenset({'A Thousand Years'}), frozenset({'ABC'}), frozenset({'Africa'}), frozenset({'Again'}), frozenset({"Ain't No Mountain High Enough"}), frozenset({"Ain't No Rest for the Wicked - Original Version"}), frozenset({'Alive'}), frozenset({'All I Want'}), frozenset({'All Me'}), frozenset({'All My Friends'}), frozenset({'All Night'}), frozenset({'All Night (feat. Knox Fortune)'}), frozenset({'All Of The Lights'}), frozenset({'All Star'}), frozenset({'All The Small Things'}), frozenset({'All The Time'}), frozenset({'All Time Low'}), frozenset({"All We Got (f

In [410]:
supportData['Mask Off']

0.0316

In [412]:
supportData[frozenset({'HUMBLE.', 'Mask Off', 'iSpy (feat. Lil Yachty)'})]

0.0101

In [413]:
supportData[frozenset({'HUMBLE.', 'Mask Off'})]

0.0204

In [409]:
L[2]

[frozenset({'Congratulations', 'HUMBLE.', 'iSpy (feat. Lil Yachty)'}),
 frozenset({'HUMBLE.', 'Mask Off', 'iSpy (feat. Lil Yachty)'}),
 frozenset({'DNA.', 'HUMBLE.', 'Mask Off'}),
 frozenset({'HUMBLE.', 'Tunnel Vision', 'XO TOUR Llif3'}),
 frozenset({'DNA.', 'HUMBLE.', 'XO TOUR Llif3'}),
 frozenset({'Congratulations', 'HUMBLE.', 'XO TOUR Llif3'}),
 frozenset({'Bad and Boujee (feat. Lil Uzi Vert)',
            'Congratulations',
            'HUMBLE.'}),
 frozenset({'Bad and Boujee (feat. Lil Uzi Vert)',
            'Bounce Back',
            'Broccoli (feat. Lil Yachty)'}),
 frozenset({'Congratulations', 'Mask Off', 'XO TOUR Llif3'}),
 frozenset({'Congratulations', 'HUMBLE.', 'Mask Off'}),
 frozenset({'HUMBLE.', 'Mask Off', 'goosebumps'}),
 frozenset({'Bad and Boujee (feat. Lil Uzi Vert)', 'HUMBLE.', 'Mask Off'}),
 frozenset({'Bounce Back', 'HUMBLE.', 'Mask Off'}),
 frozenset({'HUMBLE.', 'XO TOUR Llif3', 'goosebumps'}),
 frozenset({'HUMBLE.', 'Mask Off', 'XO TOUR Llif3'}),
 frozenset({'

In [390]:
supportData['0 To 100 / The Catch Up']

0.0161

In [391]:
supportData[frozenset({'0 To 100 / The Catch Up'})]

KeyError: ignored

In [354]:
frozenset({"'Till I Collapse"})
print(supportData[supportData[frozenset({"'Till I Collapse"})]])

TypeError: ignored

---

In [282]:
frecuencias_2=dict()
for i in combinacion_2:
  frecuencias_2[i]=0
for i in combinacion_2:
  for j in records:
    if (combinacion_2[i]<= j):
      frecuencias_2[i]+=1
frecuencias2=list(frecuencias_2.values())
frecuencias2.sort()
print(frecuencias2)
print(len(frecuencias2))

NameError: ignored

In [ ]:
set_frecuentes2=[]
for i in frecuencias_2:
  if frecuencias_2[i]>150:
    set_frecuentes2.append(list(combinacion_2[i]))
print(len(set_frecuentes2))
set_frecuentes2.sort()
set_frecuentes2

22


[['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Mask Off'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'iSpy (feat. Lil Yachty)'],
 ['Caroline', 'Broccoli (feat. Lil Yachty)'],
 ['Congratulations', 'Mask Off'],
 ['Congratulations', 'XO TOUR Llif3'],
 ['Congratulations', 'iSpy (feat. Lil Yachty)'],
 ['DNA.', 'HUMBLE.'],
 ['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)'],
 ['HUMBLE.', 'Bounce Back'],
 ['HUMBLE.', 'Congratulations'],
 ['HUMBLE.', 'Mask Off'],
 ['HUMBLE.', 'XO TOUR Llif3'],
 ['HUMBLE.', 'iSpy (feat. Lil Yachty)'],
 ['Let Me Love You', 'Closer'],
 ['No Problem (feat. Lil Wayne & 2 Chainz)', 'Broccoli (feat. Lil Yachty)'],
 ['XO TOUR Llif3', 'Mask Off'],
 ['goosebumps', 'Congratulations'],
 ['goosebumps', 'HUMBLE.'],
 ['goosebumps', 'XO TOUR Llif3']]

In [ ]:
print(set_frecuentes2[0][-1])

Bounce Back


In [ ]:
combinacion_3=dict()

for i in range(len(set_frecuentes2)):
  #print(i)
  diferentes=[]
  primer=set_frecuentes2[i][:len(set_frecuentes2[i])]
  #diferentes.append(set_frecuentes2[i][-1])
  for j in range(i+1,len(set_frecuentes2)):
    if(set_frecuentes2[i][:len(set_frecuentes2[i])-1]==set_frecuentes2[j][:len(set_frecuentes2[j])-1]):
      diferentes.append(primer+[set_frecuentes2[j][-1]])
    else:
      break
  #print(cont)
  print(diferentes)
  


[['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Caroline'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'Caroline'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Mask Off', 'iSpy (feat. Lil Yachty)']]
[]
[]
[['Congratulations', 'Mask Off', 'XO TOUR Llif3'], ['Congratulations', 'Mask Off', 'iSpy (feat. Lil Yachty)']]
[['Congratulations', 'XO TOUR Llif3', 'iSpy (feat. Lil Yachty)']]
[]
[]
[['HUMBLE

In [ ]:
combinacion_3=dict()
total=[]
for i in range(len(set_frecuentes2)):
  #print(i)
  diferentes=[]
  primer=set_frecuentes2[i][:len(set_frecuentes2[i])]
  #diferentes.append(set_frecuentes2[i][-1])
  for j in range(i+1,len(set_frecuentes2)):
    if(set_frecuentes2[i][:len(set_frecuentes2[i])-1]==set_frecuentes2[j][:len(set_frecuentes2[j])-1]):
      diferentes.append(primer+[set_frecuentes2[j][-1]])
    else:
      break
  #print(cont)
  print(diferentes)
  total+=diferentes
  

[['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Caroline'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'Caroline'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Broccoli (feat. Lil Yachty)', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline', 'iSpy (feat. Lil Yachty)']]
[['Bad and Boujee (feat. Lil Uzi Vert)', 'Mask Off', 'iSpy (feat. Lil Yachty)']]
[]
[]
[['Congratulations', 'Mask Off', 'XO TOUR Llif3'], ['Congratulations', 'Mask Off', 'iSpy (feat. Lil Yachty)']]
[['Congratulations', 'XO TOUR Llif3', 'iSpy (feat. Lil Yachty)']]
[]
[]
[['HUMBLE

In [ ]:
total

[['Bad and Boujee (feat. Lil Uzi Vert)',
  'Bounce Back',
  'Broccoli (feat. Lil Yachty)'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Caroline'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Mask Off'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Bounce Back',
  'iSpy (feat. Lil Yachty)'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Broccoli (feat. Lil Yachty)',
  'Caroline'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Broccoli (feat. Lil Yachty)',
  'Mask Off'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Broccoli (feat. Lil Yachty)',
  'iSpy (feat. Lil Yachty)'],
 ['Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline', 'Mask Off'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Caroline',
  'iSpy (feat. Lil Yachty)'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  'Mask Off',
  'iSpy (feat. Lil Yachty)'],
 ['Congratulations', 'Mask Off', 'XO TOUR Llif3'],
 ['Congratulations', 'Mask Off', 'iSpy (feat. Lil Yachty)'],
 ['Congratulations', 'XO TOUR Llif3', 'iSpy (feat. L

In [ ]:
i=0
while(i<len(set_frecuentes2)):
  #print(i)
  diferentes=[]
  cont=1
  primer=set_frecuentes2[i][:len(set_frecuentes2[i])]
  #print(primer)
  #diferentes.append(set_frecuentes2[i][-1])
  for j in range(i+1,len(set_frecuentes2)):
    if(set_frecuentes2[i][:len(set_frecuentes2[i])-1]==set_frecuentes2[j][:len(set_frecuentes2[j])-1]):
      diferentes.append(primer+[set_frecuentes2[j][-1]])
      cont+=1
    else:
      break
  i+=cont
  #print(cont)
  print(diferentes)

[['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Caroline'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back', 'iSpy (feat. Lil Yachty)']]
[]
[['Congratulations', 'Mask Off', 'XO TOUR Llif3'], ['Congratulations', 'Mask Off', 'iSpy (feat. Lil Yachty)']]
[]
[['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back'], ['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)', 'Congratulations'], ['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)', 'Mask Off'], ['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)', 'XO TOUR Llif3'], ['HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)', 'iSpy (feat. Lil Yachty)']]
[]
[]
[]
[['goosebumps', 'Congratulations', 'HUMBLE.'], ['goosebumps', 'Congratulations', 'XO TOUR Llif3']]


In [394]:
a=frozenset({'a','b'})
b={'b'}
c=a|b
list(a)

['a', 'b']